**Diplomatura en Ciencia de Datos, Aprendizaje Automático y sus Aplicaciones**

---

Mentoría #13 - Cómo hacer un Clasificador de Pliegos todoterreno (y de otros tipos de textos) usando NLP

---
Integrantes Grupo [1|2]:
*   Bruce Wayne: bruce.w@wayne-entreprises.com
*   Alfred Pennyworth: alfred@msn.com
*   Richard Grayson: robin_132@gmail.com
---

Edición 2023


# Introducción

En esta Mentoría, trabajaremos con un conjunto de datos que comprende aproximadamente 100.000 pliegos y licitaciones de diversos organismos nacionales, tanto públicos como privados. Estos datos se obtuvieron de un sistema/servicio diseñado para monitorear oportunidades de negocios, capturar la información en una base de datos, normalizarla y, posteriormente, clasificarla para informar a los usuarios según sus áreas de interés. Los usuarios de este sistema reciben alertas automáticas cada vez que se publica una oportunidad comercial que coincide con su perfil.

## Desafío

Como parte del proceso de clasificación, los pliegos se etiquetan utilizando principalmente reglas estáticas, como palabras clave, lo cual deja margen para optimizaciones. El título de cada pliego y principalmente la descripción de los objetos que se están licitando son campos de tipo texto y escritos por personas, por lo que naturalmente presentan ambigüedades y características propias del lenguaje que llevan a que un enfoque rígido y estático de clasificación, como el actual, resulte limitado y poco eficiente.

El Desafío es utilizar las técnicas de Aprendizaje Automático, logrando así un "Clasificador" que utilice técnicas de Procesamiento de Lenguaje Natural (NLP) para clasificar de manera más eficiente en qué rubro o categoría se encuentra un pliego, basándose en su texto descriptivo y otros campos relacionados.

## Interés General

Más allá de la aplicación específica en este conjunto de datos, este problema, al estar vinculado al Procesamiento de Lenguaje Natural (PLN) y la clasificación, tiene la ventaja de poder ser utilizado posteriormente para otros tipos de contenido. De esta manera, el clasificador desarrollado podría aplicarse para categorizar libros, noticias, textos, tweets, publicaciones, artículos, etc.

## Descripción del dataset

A continuación se enumeran las diferentes variables del dataset, así como una breve descripción de su significado:

- **id**: Clave única y primaria autoincremental de la tabla;
- **cargado**: Fecha de carga del pliego;
- **idexterno**: Id del pliego en la fuente;
- **referencia**: Campo auxiliar obtenido de la fuente;
- **objeto**: Campo principal, descripción del producto o servicio objeto de la licitación;
- **rubro**: Campo de categorización disponible en la fuente. No siempre está disponible;
- **agencia**: Empresa o Ente que lanza la licitación;
- **apertura**: Fecha de apertura del pliego (vencimiento para presentarse al pliego);
- **subrubro**: Subcategoría del pliego (también obtenido desde la fuente);
- **pais**: País donde se lanza la licitación;
- **observaciones**: Campo auxiliar donde se guardan datos extra que puede variar según la fuente;
- **monto**: Monto del pliego, no siempre está publicado;
- **divisaSimboloISO**: Moneda en la que se especifica el pliego;
- **visible**: campo binario que determina si el pliego va a ser visualizado por el sistema (True/False);
- **categoría**: Tipo de pliego, categorización entre diversos tipos de pliegos (Compra Directa, Licitación Simple, Subasta, etc.);
- **fuente**: Fuente de donde se obtuvo la licitación.

# Trabajo Práctico #3

El propósito de este último trabajo práctico es poder hacer una aproximación inicial a los conceptos del aprendizaje supervisado.

El objetivo no es alcanzar el modelo definitivo con sus parámetros óptimos, sino más bien adquirir práctica en la ejecución de los pasos esenciales dentro de un proceso de aprendizaje supervisado.

Desde comprender la pregunta de investigación hasta la preparación de los datos, desde la segmentación del conjunto de datos hasta la evaluación del modelo. Para llevar a cabo esta actividad, emplearemos los conjuntos de datos generados en el TP2

A continuación se enumeran los ejercicios propuestos como guía:

## Ejercicio 1: Limpieza de Datos y Variables
A pesar que mucho del trabajo de preparación de los datos ya se realizó en los TPs anteriores, aún falta preparar los datos para poder alimentar un modelo de clasificación. Aplicar técnicas de transformación, encoding y escalado (si hace falta).

## Ejercicio 2: Testing de Modelos

Para modelar una clasificación multi-clase existen diversas opciones. Explorar modelos de clasificación binaria aplicando técnicas OvR, OvO, etc. Elegir 3 modelos diferentes y compararlos.

Explorar también otros modelos que tengan múltiples salidas como decision trees, y sus técnicas derivadas (como random forest por ejemplo).

La idea es hacer un primer "barrido" con diferentes modelos con todos los hiperparámetros con sus valores por default, que permita evaluar qué algoritmo se ajusta mejor al problema, para luego pasar (en el siguiente ejercicio) a una optimización del mejor modelo ajustando los hiperparámetros.

## Ejercicio 3: Optimización de Hiperparámetros

Evaluar las distintas métricas (precision, recall, f1-score y support) para visualizar la performance del modelo. Armar la matriz de confusión y contrastar los resultados del modelo ya optimizado con los primeros resultados obtenidos con los primeros modelos testeados.

## Ejercicio 4: Conclusión
Con todo el recorrido realizado, elaborar una conclusión final sobre la mentoría. Dentro de la conclusión, considerar lo que se ha aprendido en las diferentes entregas (TP1 y TP2).

Parte de estas conclusiones les servirán también para el video final donde se presente el caso.

El video que sigue a este práctico serviría como informe de resultados y observaciones de este práctico, resaltando aprendizajes y problemáticas con las que se hayan encontrado.

Recuerden que un buen resultado no implica necesariamente encontrar el mejor algoritmo, sino poder asimilar un proceso de modelado completo, desde el dataset bruto hasta la optimización final.

# Desarrollo

In [1]:
# Link al dataset: https://drive.google.com/file/d/1jo9cZNHAPpsTosW-2JA4CLo4SaFCHfIs/view?usp=drive_link

In [2]:
import pandas as pd
import missingno as msno
import matplotlib.pyplot as plt
import numpy as np

pd.set_option('display.max_rows', None)

df = pd.read_csv("./dataset_concatenado.csv" , encoding='latin-1')

In [3]:
df.head()

,index,referencia,objeto,rubro,agencia,apertura,monto,fuente
0,347,ContrataciÃ³n Directa 464/2017,"['bols', 'aliment']","['agric', 'ganaderia', 'caza', 'silvicult']","['com', 'nacional', 'energi', 'atom']",2017-09-29,0.0,Argentinacompra
1,348,Subasta PÃºblica 48/2017,"['cabez', 'gan', 'vacun', 'lot', 'novill', 'go...","['agric', 'ganaderia', 'caza', 'silvicult']","['mayor', 'general', 'ejercit']",2017-10-26,0.0,Argentinacompra
2,349,Subasta PÃºblica 47/2017,"['cabez', 'gan', 'vacun', 'lot', 'novill', 'go...","['agric', 'ganaderia', 'caza', 'silvicult']","['mayor', 'general', 'ejercit']",2017-10-24,0.0,Argentinacompra
3,350,Subasta PÃºblica 46/2017,"['cabez', 'gan', 'vacun', 'lot', 'vac', 'inver...","['agric', 'ganaderia', 'caza', 'silvicult']","['mayor', 'general', 'ejercit']",2017-10-19,0.0,Argentinacompra
4,351,Subasta PÃºblica 45/2017,"['cabez', 'gan', 'vacun', 'lot', 'novill', 'go...","['agric', 'ganaderia', 'caza', 'silvicult']","['mayor', 'general', 'ejercit']",2017-10-12,0.0,Argentinacompra


In [4]:
!pip install unidecode

In [5]:
#from unidecode import unidecode

#df['referencia'] = df['referencia'].apply(unidecode)
#df['objeto'] = df['objeto'].apply(lambda x: [unidecode(word) for word in x])
#df['rubro'] = df['rubro'].apply(unidecode)


In [6]:
from sklearn.feature_extraction.text import TfidfVectorizer

# Tokenización y vectorización de 'objeto'
tfidf_vectorizer = TfidfVectorizer()
#objeto_tfidf = tfidf_vectorizer.fit_transform(df['objeto'].apply(lambda x: ' '.join(x)))

# Vectorización de 'rubro'
#rubro_tfidf = tfidf_vectorizer.fit_transform(df['rubro'])

# Encoding de etiquetas de 'agencia' (ejemplo de Label Encoding)
from sklearn.preprocessing import LabelEncoder
label_encoder = LabelEncoder()
df['agencia_encoded'] = label_encoder.fit_transform(df['agencia'])
df['rubro_encoded'] = label_encoder.fit_transform(df['rubro'])
df['objeto_encoded'] = label_encoder.fit_transform(df['objeto'])


In [7]:
df['agencia_encoded']

0          194
1         1024
2         1024
3         1024
4         1024
5         1024
6         1024
7         1023
8         1031
9         1108
10        1108
11        1108
12        1108
13        1069
14         936
15         936
16        1227
17        1210
18        1210
19        1023
20        1023
21        1204
22        1023
23         678
24        1023
25        1023
26        1023
27        1023
28        1023
29        1023
30         678
31        1025
32        1025
33         936
34        1023
35        1201
36         957
37        1201
38        1201
39         570
40         936
41        1024
42        1024
43           6
44         936
45        1024
46         936
47        1023
48        1024
49         936
50         936
51        1022
52         965
53         936
54        1023
55        1023
56         734
57         194
58         194
59           3
60           5
61           5
62        1025
63        1023
64         958
65        1024
66        

In [20]:
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, classification_report

# Dividir los datos en características (X) y etiquetas (y)
X = df[['agencia_encoded', 'objeto_encoded','monto']]  # Ajusta las características según tus necesidades
y = df['rubro_encoded']  # Ajusta 'clase' según la columna de etiquetas en tu dataset

# Dividir los datos en conjuntos de entrenamiento y prueba
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=13)
X_train/=np.amax(X_train,axis=0)
X_test/=np.amax(X_test,axis=0)
#y_train/=np.amax(y_train)
#y_test/=np.amax(y_test)

print(y_test)


# # Crear y entrenar el modelo Random Forest
# rf_model = RandomForestClassifier(n_estimators=10, random_state=13)
# rf_model.fit(X_train, y_train)

# # Realizar predicciones en el conjunto de prueba
# y_pred = rf_model.predict(X_test)

# # Evaluar el rendimiento del modelo
# accuracy = accuracy_score(y_test, y_pred)
# report = classification_report(y_test, y_pred)

# print("Accuracy:", accuracy)
# print("Classification Report:\n", report)


62785      711
43662     4843
14813     4885
59648     4242
37456      355
49170     4751
51424     4769
98932     4617
78849     4843
21953     4769
94740     3863
8331      2695
56349     4769
65674      100
97614     4769
97882      711
32028     4276
81177      506
35695     4843
48646      239
93377     4991
61533     4769
43045     5056
96909     4769
71152     1009
25495     2692
43205     4769
59164       23
60113     3584
58462     4669
36232     2017
96275     2017
86970     4276
28260       33
79399     2017
20996     4769
4493      3586
36605     4276
100823    5089
92089      385
27794      711
57552     4276
84528     4825
35627     1852
96623     3481
2964      4622
51153     4276
34687     2017
53161     4608
48976     4769
58407     4769
35820     3584
87823      239
60741     4769
99235      407
39205     4428
101078    5089
85862     4843
32591     3708
81978     4276
24943     2017
9476      4843
28380     4769
95307     4276
5509      3708
95623     1821
40220     

In [21]:
from sklearn.tree import DecisionTreeClassifier


tree_ = DecisionTreeClassifier(max_depth=3, min_samples_leaf=3, random_state=13)
tree_.fit(X_train, y_train)
y_pred = tree_.predict(X_test)

accuracy = accuracy_score(y_test, y_pred)
report = classification_report(y_test, y_pred)

print("Accuracy:", accuracy)
print("Classification Report:\n", report)

Accuracy: 0.16953570725949244
Classification Report:
               precision    recall  f1-score   support

           1       0.00      0.00      0.00         1
           3       0.00      0.00      0.00         2
           4       0.00      0.00      0.00         1
           5       0.00      0.00      0.00        71
           6       0.00      0.00      0.00         2
           7       0.00      0.00      0.00         3
          15       0.00      0.00      0.00         1
          16       0.00      0.00      0.00        11
          17       0.00      0.00      0.00        39
          18       0.00      0.00      0.00         1
          21       0.00      0.00      0.00         2
          22       0.00      0.00      0.00        18
          23       0.00      0.00      0.00       547
          24       0.00      0.00      0.00         5
          25       0.00      0.00      0.00         1
          30       0.00      0.00      0.00        12
          33       0.00    

/home/santiago/anaconda3/lib/python3.11/site-packages/sklearn/metrics/_classification.py:1469: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/home/santiago/anaconda3/lib/python3.11/site-packages/sklearn/metrics/_classification.py:1469: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/home/santiago/anaconda3/lib/python3.11/site-packages/sklearn/metrics/_classification.py:1469: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [25]:
import tensorflow as tf
from tensorflow import keras
from sklearn.preprocessing import StandardScaler

# Escalar las características (esto puede variar según las características)
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

# Crear una arquitectura de red neuronal simple
model = keras.Sequential([
    keras.layers.Input(shape=(X_train_scaled.shape[1],)),
    keras.layers.Dense(64, activation='relu'),
    keras.layers.Dense(32, activation='relu'),
    keras.layers.Dense(1, activation='softmax')  # Ajusta el número de clases
])

# Compilar el modelo
model.compile(optimizer='adam',
              loss='CategoricalCrossentropy',
              metrics=['accuracy'])

print(model.get_weights())

# # Entrenar la red neuronal
# model.fit(X_train_scaled, y_train, epochs=10, batch_size=32, validation_split=0.2)

# # Evaluar el rendimiento del modelo en el conjunto de prueba
# test_loss, test_accuracy = model.evaluate(X_test_scaled, y_test)
# print("Test Accuracy:", test_accuracy)


[array([[ 0.03794128,  0.01202855,  0.23223156, -0.0055666 , -0.160919  ,
        -0.09269772,  0.04879716, -0.19582537,  0.02139646,  0.06558678,
        -0.13996674,  0.15156057, -0.2493044 , -0.07401647,  0.07665381,
         0.01939973, -0.22615458,  0.22419238, -0.0892024 ,  0.19427782,
         0.21342242, -0.05262822,  0.10486653,  0.04778159, -0.12155505,
        -0.22638454,  0.23095357, -0.06423123, -0.25709888, -0.13160005,
         0.02155006, -0.15122886,  0.06037506, -0.12758611, -0.04794434,
        -0.06310038,  0.2685545 ,  0.0942513 , -0.20361471, -0.08239871,
         0.2825939 ,  0.20587379, -0.19344392, -0.26003882, -0.01123679,
         0.27341282, -0.26018316, -0.29452905, -0.29731157,  0.01146838,
        -0.13379341,  0.2824365 , -0.22272256,  0.1099093 ,  0.00811341,
         0.20269698, -0.24263178, -0.08341271, -0.08550197,  0.10283533,
        -0.22452779, -0.25119495, -0.27893683, -0.18883795],
       [-0.18899179,  0.01278731,  0.18425408,  0.23728359,  0